# Sales Email Generator with Salesforce Context

This notebook demonstrates a generative AI-powered sales email generator that:
- Retrieves relevant context from Salesforce (Lead/Opportunity data, past activities)
- Uses RAG (Retrieval-Augmented Generation) to ground emails in real data
- Applies brand style guidelines and guardrails
- Validates consent and compliance requirements

**Note:** This demo uses mock data to simulate Salesforce API responses.

In [ ]:
import json
import re
from datetime import datetime, timedelta
from typing import Dict, List, Optional
import random

## 1. Mock Salesforce Data

Simulating data that would come from Salesforce APIs (Lead, Opportunity, Account, Activities)

In [ ]:
# Mock Lead/Opportunity Data
MOCK_LEADS = [
    {
        "Id": "00Q5e000001aBcD",
        "FirstName": "Sarah",
        "LastName": "Chen",
        "Email": "sarah.chen@techcorp.com",
        "Company": "TechCorp Industries",
        "Title": "VP of Engineering",
        "Industry": "Technology",
        "LeadSource": "Web",
        "Status": "Working - Contacted",
        "Rating": "Hot",
        "EmailOptOut": False,
        "ConsentGiven__c": True,
        "Description": "Downloaded whitepaper on AI integration. Interested in automating customer support."
    },
    {
        "Id": "00Q5e000001aBcE",
        "FirstName": "Michael",
        "LastName": "Rodriguez",
        "Email": "m.rodriguez@financeco.com",
        "Company": "FinanceCo Solutions",
        "Title": "Director of Operations",
        "Industry": "Financial Services",
        "LeadSource": "Event",
        "Status": "Open - Not Contacted",
        "Rating": "Warm",
        "EmailOptOut": False,
        "ConsentGiven__c": True,
        "Description": "Met at Dreamforce. Exploring Service Cloud for compliance tracking."
    }
]

MOCK_OPPORTUNITIES = [
    {
        "Id": "006abc123",
        "Name": "TechCorp - Service Cloud Implementation",
        "AccountId": "001xyz789",
        "StageName": "Proposal/Price Quote",
        "Amount": 85000,
        "CloseDate": "2025-12-15",
        "Probability": 60,
        "Type": "New Business",
        "LeadSource": "Web",
        "Description": "AI-powered customer support automation for 200-agent contact center"
    }
]

# Mock Activity History
MOCK_ACTIVITIES = [
    {
        "Id": "00T123abc",
        "WhoId": "00Q5e000001aBcD",
        "Type": "Email",
        "Subject": "RE: AI Integration Whitepaper",
        "ActivityDate": "2025-10-13",
        "Status": "Completed",
        "Description": "Sarah expressed interest in a demo. Mentioned pain points with current ticketing system."
    },
    {
        "Id": "00T123abd",
        "WhoId": "00Q5e000001aBcD",
        "Type": "Call",
        "Subject": "Discovery Call",
        "ActivityDate": "2025-10-15",
        "Status": "Completed",
        "Description": "30-min call. Key requirements: reduce AHT by 30%, integrate with existing CRM, AI-powered case routing."
    }
]

# Mock Similar Won Opportunities (for context)
MOCK_SIMILAR_WINS = [
    {
        "AccountName": "GlobalTech Inc",
        "Industry": "Technology",
        "Amount": 120000,
        "WinReason": "Superior AI capabilities, seamless Slack integration, 40% AHT reduction in POC",
        "UseCase": "Customer support automation with Einstein Bots"
    },
    {
        "AccountName": "InnovateSoft",
        "Industry": "Technology",
        "Amount": 95000,
        "WinReason": "Proven ROI calculator showed 18-month payback, strong references from similar-sized companies",
        "UseCase": "Service Cloud for technical support team with AI-powered knowledge base"
    }
]

print("✅ Mock Salesforce data loaded")
print(f"   - {len(MOCK_LEADS)} Leads")
print(f"   - {len(MOCK_OPPORTUNITIES)} Opportunities")
print(f"   - {len(MOCK_ACTIVITIES)} Activities")
print(f"   - {len(MOCK_SIMILAR_WINS)} Similar Won Deals")

## 2. Brand Style Guidelines & Guardrails

Define company voice, tone, and compliance rules

In [ ]:
BRAND_GUIDELINES = {
    "voice": "professional, consultative, solution-focused",
    "tone": "confident but not pushy, empathetic to customer challenges",
    "prohibited_phrases": [
        "guaranteed results",
        "best in the industry",
        "revolutionary",
        "you need to",
        "limited time offer"
    ],
    "required_elements": [
        "personalization based on prospect context",
        "clear call-to-action",
        "value proposition tied to their needs"
    ],
    "max_length_words": 250,
    "signature": "Best regards,\nJohn Smith\nSenior Account Executive\nSalesforce\nphone: (555) 123-4567\nemail: john.smith@salesforce.com"
}

COMPLIANCE_RULES = {
    "require_consent": True,
    "require_opt_in": True,
    "include_unsubscribe": True,
    "gdpr_compliant": True
}

print("✅ Brand guidelines and compliance rules configured")

## 3. Context Retrieval Service (Simulated RAG)

Retrieve relevant information about the lead/opportunity

In [ ]:
class ContextRetriever:
    """Simulates RAG-style context retrieval from Salesforce"""
    
    def __init__(self, leads, opportunities, activities, similar_wins):
        self.leads = leads
        self.opportunities = opportunities
        self.activities = activities
        self.similar_wins = similar_wins
    
    def get_lead_context(self, lead_id: str) -> Dict:
        """Retrieve comprehensive context for a lead"""
        lead = next((l for l in self.leads if l['Id'] == lead_id), None)
        if not lead:
            return {"error": "Lead not found"}
        
        # Get related activities
        related_activities = [a for a in self.activities if a['WhoId'] == lead_id]
        
        # Get similar wins by industry (simulate embedding similarity)
        similar_wins = [w for w in self.similar_wins if w['Industry'] == lead.get('Industry')]
        
        return {
            "lead": lead,
            "activities": related_activities,
            "similar_wins": similar_wins[:2],  # Top 2 most similar
            "context_summary": self._generate_summary(lead, related_activities)
        }
    
    def _generate_summary(self, lead: Dict, activities: List[Dict]) -> str:
        """Generate a text summary of the lead context"""
        summary_parts = []
        
        summary_parts.append(f"Contact: {lead['FirstName']} {lead['LastName']}, {lead['Title']} at {lead['Company']}")
        summary_parts.append(f"Industry: {lead['Industry']}")
        summary_parts.append(f"Status: {lead['Status']} | Rating: {lead['Rating']}")
        
        if lead.get('Description'):
            summary_parts.append(f"Context: {lead['Description']}")
        
        if activities:
            summary_parts.append(f"\nRecent Interactions ({len(activities)}):")
            for activity in activities[-3:]:  # Last 3 activities
                summary_parts.append(f"  - {activity['ActivityDate']}: {activity['Subject']} - {activity['Description']}")
        
        return "\n".join(summary_parts)

retriever = ContextRetriever(MOCK_LEADS, MOCK_OPPORTUNITIES, MOCK_ACTIVITIES, MOCK_SIMILAR_WINS)
print("✅ Context retriever initialized")

## 4. Prompt Template System

Structured prompts for different email types (stored as Custom Metadata in real implementation)

In [ ]:
PROMPT_TEMPLATES = {
    "follow_up_after_call": {
        "name": "Follow-up After Discovery Call",
        "version": "1.2",
        "template": """You are writing a follow-up email after a discovery call with a prospective customer.

CONTEXT:
{context}

SIMILAR SUCCESS STORIES:
{similar_wins}

BRAND GUIDELINES:
- Voice: {voice}
- Tone: {tone}
- AVOID these phrases: {prohibited_phrases}
- Maximum length: {max_length} words

Write a personalized follow-up email that:
1. References specific points from the discovery call
2. Addresses their key pain points with relevant solutions
3. Mentions a similar customer success story from their industry
4. Proposes a clear next step (demo, ROI analysis, or technical deep-dive)
5. Maintains the brand voice and avoids prohibited phrases

OUTPUT FORMAT:
Return JSON with:
{{
  "subject": "email subject line",
  "body": "email body text",
  "call_to_action": "specific next step",
  "citations": ["list of sources referenced"]
}}
"""
    },
    "initial_outreach": {
        "name": "Initial Outreach",
        "version": "1.0",
        "template": """You are writing an initial outreach email to a new lead.

CONTEXT:
{context}

BRAND GUIDELINES:
- Voice: {voice}
- Tone: {tone}
- AVOID: {prohibited_phrases}
- Maximum: {max_length} words

Write a brief, personalized introduction that:
1. References how they came to our attention (lead source, activity)
2. Acknowledges their industry/role
3. Offers specific value (not generic)
4. Suggests a low-commitment next step

OUTPUT FORMAT: JSON with subject, body, call_to_action, citations
"""
    }
}

print("✅ Prompt templates loaded:")
for key, template in PROMPT_TEMPLATES.items():
    print(f"   - {template['name']} (v{template['version']})")

## 5. Email Generator with Validation

Main service that orchestrates context retrieval, prompt construction, and validation

In [ ]:
class EmailGenerator:
    """Main email generation service with guardrails"""
    
    def __init__(self, retriever, brand_guidelines, compliance_rules):
        self.retriever = retriever
        self.brand_guidelines = brand_guidelines
        self.compliance_rules = compliance_rules
        self.generation_log = []  # Simulates LLM_Call_Log__c
    
    def validate_consent(self, lead: Dict) -> tuple[bool, str]:
        """Check consent and opt-in status"""
        if lead.get('EmailOptOut', False):
            return False, "Lead has opted out of email communications"
        
        if self.compliance_rules['require_consent'] and not lead.get('ConsentGiven__c', False):
            return False, "Consent not given for marketing communications"
        
        return True, "Consent validated"
    
    def validate_content(self, email_body: str) -> tuple[bool, List[str]]:
        """Validate email content against brand guidelines"""
        violations = []
        
        # Check prohibited phrases
        for phrase in self.brand_guidelines['prohibited_phrases']:
            if phrase.lower() in email_body.lower():
                violations.append(f"Contains prohibited phrase: '{phrase}'")
        
        # Check length
        word_count = len(email_body.split())
        if word_count > self.brand_guidelines['max_length_words']:
            violations.append(f"Exceeds max length: {word_count} words (max: {self.brand_guidelines['max_length_words']})")
        
        return len(violations) == 0, violations
    
    def construct_prompt(self, template_key: str, context: Dict) -> str:
        """Build the full prompt from template and context"""
        template = PROMPT_TEMPLATES[template_key]['template']
        
        # Format similar wins
        similar_wins_text = "\n".join([
            f"- {win['AccountName']} ({win['Industry']}): {win['UseCase']}. {win['WinReason']}"
            for win in context.get('similar_wins', [])
        ])
        
        prompt = template.format(
            context=context['context_summary'],
            similar_wins=similar_wins_text or "No similar wins available",
            voice=self.brand_guidelines['voice'],
            tone=self.brand_guidelines['tone'],
            prohibited_phrases=", ".join(self.brand_guidelines['prohibited_phrases']),
            max_length=self.brand_guidelines['max_length_words']
        )
        
        return prompt
    
    def simulate_llm_call(self, template_key: str, lead: Dict) -> Dict:
        """Simulate an LLM API call with realistic output"""
        # In real implementation, this would call OpenAI/Cohere/etc via Named Credentials
        
        if template_key == "follow_up_after_call" and lead['Id'] == "00Q5e000001aBcD":
            return {
                "subject": "Following up on our conversation - AI-powered support automation",
                "body": """Hi Sarah,

Thank you for the insightful conversation on Tuesday about TechCorp's customer support challenges. I appreciated you sharing the specific pain points around average handle time and the difficulties your agents face with the current ticketing system.

Based on what you described—particularly the goal to reduce AHT by 30% and improve case routing—I think you'd find value in how we helped GlobalTech Inc tackle similar challenges. They implemented our AI-powered Service Cloud solution and achieved a 40% reduction in AHT within their first quarter, largely through intelligent case routing and Einstein Bot deflection.

I've prepared a brief demo focusing specifically on:
• AI-powered case routing that learns from your top performers
• Integration with your existing CRM (as you mentioned this was critical)
• Real-time agent assist with knowledge article suggestions

Would next Wednesday or Thursday work for a 30-minute technical demo? I can also bring our solutions engineer to address any integration questions your team might have.

Looking forward to showing you what's possible.""",
                "call_to_action": "Schedule 30-minute technical demo for next week",
                "citations": [
                    "Discovery call notes from 2025-10-15",
                    "GlobalTech Inc case study (similar industry)",
                    "Lead description: AI integration whitepaper download"
                ]
            }
        elif template_key == "initial_outreach" and lead['Id'] == "00Q5e000001aBcE":
            return {
                "subject": "Great meeting you at Dreamforce - Service Cloud for compliance",
                "body": """Hi Michael,

It was a pleasure meeting you at Dreamforce last week. I enjoyed our conversation about the unique compliance tracking challenges in financial services.

Given your interest in Service Cloud for compliance workflows, I thought you might appreciate seeing how InnovateSoft (also in financial services) streamlined their audit trail requirements while improving customer response times.

I'd love to share a quick overview of how we can help FinanceCo Solutions maintain compliance while scaling your operations team. Would you have 15 minutes next week for a brief call?

No pressure—just thought this might be timely given what you shared about your Q4 initiatives.""",
                "call_to_action": "Schedule 15-minute introductory call",
                "citations": [
                    "Lead source: Event (Dreamforce)",
                    "Lead description: Exploring Service Cloud for compliance",
                    "InnovateSoft win (financial services)"
                ]
            }
        
        return {
            "subject": "Following up on your interest",
            "body": "Generic email body...",
            "call_to_action": "Schedule a call",
            "citations": []
        }
    
    def generate_email(self, lead_id: str, template_key: str) -> Dict:
        """Main method: generate and validate email"""
        start_time = datetime.now()
        
        # Step 1: Retrieve context
        context = self.retriever.get_lead_context(lead_id)
        if 'error' in context:
            return {"error": context['error']}
        
        lead = context['lead']
        
        # Step 2: Validate consent
        consent_valid, consent_msg = self.validate_consent(lead)
        if not consent_valid:
            return {"error": consent_msg, "compliance_blocked": True}
        
        # Step 3: Construct prompt
        prompt = self.construct_prompt(template_key, context)
        
        # Step 4: Call LLM (simulated)
        llm_response = self.simulate_llm_call(template_key, lead)
        
        # Step 5: Validate content
        content_valid, violations = self.validate_content(llm_response['body'])
        
        if not content_valid:
            # In production: retry with adjusted prompt or use fallback
            llm_response['warnings'] = violations
        
        # Step 6: Add signature and unsubscribe
        full_body = llm_response['body'] + "\n\n" + self.brand_guidelines['signature']
        if self.compliance_rules['include_unsubscribe']:
            full_body += "\n\n---\nTo unsubscribe from future emails, click here."
        
        # Step 7: Log the generation (simulates LLM_Call_Log__c)
        duration_ms = (datetime.now() - start_time).total_seconds() * 1000
        log_entry = {
            "timestamp": datetime.now().isoformat(),
            "lead_id": lead_id,
            "template": template_key,
            "template_version": PROMPT_TEMPLATES[template_key]['version'],
            "model": "gpt-4",  # Simulated
            "tokens_used": 450,  # Simulated
            "cost_usd": 0.0135,  # Simulated
            "duration_ms": duration_ms,
            "consent_valid": consent_valid,
            "content_valid": content_valid,
            "violations": violations
        }
        self.generation_log.append(log_entry)
        
        return {
            "success": True,
            "email": {
                "to": lead['Email'],
                "to_name": f"{lead['FirstName']} {lead['LastName']}",
                "subject": llm_response['subject'],
                "body": full_body,
                "call_to_action": llm_response['call_to_action']
            },
            "metadata": {
                "citations": llm_response.get('citations', []),
                "warnings": violations,
                "log_id": len(self.generation_log) - 1
            }
        }

generator = EmailGenerator(retriever, BRAND_GUIDELINES, COMPLIANCE_RULES)
print("✅ Email generator initialized")

## 6. Generate Sample Emails

Let's generate emails for our mock leads

In [ ]:
# Example 1: Follow-up email after discovery call
print("=" * 80)
print("EXAMPLE 1: Follow-up After Discovery Call")
print("=" * 80)

result1 = generator.generate_email(
    lead_id="00Q5e000001aBcD",
    template_key="follow_up_after_call"
)

if result1.get('success'):
    email = result1['email']
    print(f"\nTo: {email['to_name']} <{email['to']}>")
    print(f"Subject: {email['subject']}")
    print("\n" + "-" * 80)
    print(email['body'])
    print("-" * 80)
    print(f"\n📌 Call to Action: {email['call_to_action']}")
    print(f"\n📚 Citations:")
    for citation in result1['metadata']['citations']:
        print(f"   - {citation}")
    
    if result1['metadata']['warnings']:
        print(f"\n⚠️  Warnings:")
        for warning in result1['metadata']['warnings']:
            print(f"   - {warning}")
else:
    print(f"❌ Error: {result1.get('error')}")

In [ ]:
# Example 2: Initial outreach email
print("\n" + "=" * 80)
print("EXAMPLE 2: Initial Outreach Email")
print("=" * 80)

result2 = generator.generate_email(
    lead_id="00Q5e000001aBcE",
    template_key="initial_outreach"
)

if result2.get('success'):
    email = result2['email']
    print(f"\nTo: {email['to_name']} <{email['to']}>")
    print(f"Subject: {email['subject']}")
    print("\n" + "-" * 80)
    print(email['body'])
    print("-" * 80)
    print(f"\n📌 Call to Action: {email['call_to_action']}")
    print(f"\n📚 Citations:")
    for citation in result2['metadata']['citations']:
        print(f"   - {citation}")
else:
    print(f"❌ Error: {result2.get('error')}")

## 7. Analytics & Monitoring

View generation logs (simulates CRM Analytics dashboard reading from LLM_Call_Log__c)

In [ ]:
import pandas as pd

# Convert logs to DataFrame
if generator.generation_log:
    df_logs = pd.DataFrame(generator.generation_log)
    
    print("EMAIL GENERATION ANALYTICS")
    print("=" * 80)
    print(f"\nTotal Emails Generated: {len(df_logs)}")
    print(f"Total Cost: ${df_logs['cost_usd'].sum():.4f}")
    print(f"Total Tokens: {df_logs['tokens_used'].sum():,}")
    print(f"Average Duration: {df_logs['duration_ms'].mean():.2f}ms")
    print(f"\nConsent Valid Rate: {df_logs['consent_valid'].sum() / len(df_logs) * 100:.1f}%")
    print(f"Content Valid Rate: {df_logs['content_valid'].sum() / len(df_logs) * 100:.1f}%")
    
    print("\n\nDetailed Log:")
    print(df_logs[['timestamp', 'template', 'model', 'tokens_used', 'cost_usd', 'duration_ms', 'content_valid']].to_string(index=False))
else:
    print("No generation logs available")

## 8. Testing Guardrails

Demonstrate compliance blocking and content validation

In [ ]:
# Test 1: Lead with email opt-out
print("TEST 1: Lead with Email Opt-Out")
print("=" * 80)

opted_out_lead = MOCK_LEADS[0].copy()
opted_out_lead['Id'] = "TEST_001"
opted_out_lead['EmailOptOut'] = True
MOCK_LEADS.append(opted_out_lead)

result_blocked = generator.generate_email(
    lead_id="TEST_001",
    template_key="initial_outreach"
)

if 'error' in result_blocked:
    print(f"✅ Correctly blocked: {result_blocked['error']}")
    if result_blocked.get('compliance_blocked'):
        print("   Reason: Compliance violation")
else:
    print("❌ Should have been blocked!")

# Test 2: Content validation
print("\n\nTEST 2: Content Validation")
print("=" * 80)

test_content = "This is a guaranteed results solution with a limited time offer that is the best in the industry!"
is_valid, violations = generator.validate_content(test_content)

print(f"Content: '{test_content}'")
print(f"\nValid: {is_valid}")
if violations:
    print("\nViolations detected:")
    for v in violations:
        print(f"  ❌ {v}")

# Clean up test lead
MOCK_LEADS = [l for l in MOCK_LEADS if l['Id'] != "TEST_001"]

## 9. Production Deployment Considerations

In a real Salesforce implementation, this would involve:

### Apex/Platform
- **Named Credentials**: Secure connection to OpenAI/Cohere APIs
- **Custom Metadata Types**: Store prompt templates with versioning
- **Custom Object** `LLM_Call_Log__c`: Track all generations with fields:
  - `Lead__c`, `Opportunity__c` (lookups)
  - `Model__c`, `PromptVersion__c`, `Tokens__c`, `Cost__c`
  - `DurationMs__c`, `ConsentValid__c`, `ContentValid__c`
- **Apex Service Classes**:
  - `EmailGeneratorService`: Main orchestration
  - `ContextRetrieverService`: RAG implementation
  - `GuardrailValidator`: Content & consent checks
- **Queueable Jobs**: Handle retries with exponential backoff for 429/5xx errors
- **Platform Cache**: Cache frequently-accessed context and similar wins

### LWC/UI
- **LWC Component** on Lead/Opportunity page:
  - Button: "Generate Email"
  - Template selector dropdown
  - Preview pane with edit capability
  - Citation display
  - One-click send to Email Composer

### Integration
- **Vector Database**: Heroku Postgres with pgvector for semantic search
- **Embeddings Pipeline**: Batch Apex job to embed Knowledge articles, win stories
- **RAG Service**: Heroku/Functions endpoint for k-NN search

### Governance
- **Approval Flow**: Prompt template changes require manager approval
- **Custom Object** `Prompt_Version__c`: Audit trail of all template changes
- **Permission Sets**: Control who can generate/edit emails
- **CRM Analytics Dashboard**: Cost, latency, success rate by user/team

### Security
- **Shield Platform Encryption**: Protect API keys
- **Protected Custom Metadata**: Store sensitive configuration
- **Field-level Security**: Control access to LLM logs
- **OAuth/JWT**: Secure Heroku↔Salesforce communication

## Summary

This notebook demonstrated:

✅ **Context Retrieval (RAG)**: Grounding emails in real Salesforce data (leads, activities, similar wins)  
✅ **Prompt Templates**: Reusable, versioned templates for different email types  
✅ **Brand Guardrails**: Validation against prohibited phrases and style guidelines  
✅ **Compliance Checks**: Consent validation before generation  
✅ **Citations**: Traceable sources for all generated content  
✅ **Observability**: Logging cost, latency, and quality metrics  
✅ **Production-Ready Patterns**: Error handling, validation, and monitoring  

**Key Salesforce Integration Points:**
- Named Credentials for secure API access
- Custom Metadata for prompt template storage
- Custom Objects for logging and analytics
- LWC components for user interface
- Queueable/Batch Apex for scalability
- CRM Analytics for monitoring

This approach ensures emails are personalized, compliant, and grounded in actual customer data while maintaining cost control and quality standards.

## 🎯 Launch Interactive Demo

Click the cell below to open the full interactive presentation showcasing this email generation system.

In [1]:
import webbrowser
import os

# Path to the demo HTML file
demo_path = os.path.abspath('../slides/sales_email_generator_demo.html')

# Open in default browser
webbrowser.open('file://' + demo_path)

print(f"🎉 Opening Sales Email Generator Demo...")
print(f"📂 File: {demo_path}")
print("\n✨ Demo Features:")
print("   • 8 interactive slides")
print("   • Live email generation demo with 2 leads and 2 email types")
print("   • RAG context retrieval visualization")
print("   • Brand guardrails and compliance workflows")
print("   • Real-time analytics dashboard")
print("   • Conservative ROI calculations")
print("\n🎯 Navigate with: Arrow keys, Previous/Next buttons, or Restart")

🎉 Opening Sales Email Generator Demo...
📂 File: /home/bbrelin/src/repos/salesforce/slides/sales_email_generator_demo.html

✨ Demo Features:
   • 8 interactive slides
   • Live email generation demo with 2 leads and 2 email types
   • RAG context retrieval visualization
   • Brand guardrails and compliance workflows
   • Real-time analytics dashboard
   • Conservative ROI calculations

🎯 Navigate with: Arrow keys, Previous/Next buttons, or Restart
